In [1]:
# ACCESS_KEY = ""
# SECRET_KEY = ""
ENCODED_SECRET_KEY = SECRET_KEY.replace("/", "%2F")
# AWS_BUCKET_NAME = ""
MOUNT_NAME = "yelp-mount"

dbutils.fs.mount("s3a://%s:%s@%s" % (ACCESS_KEY, ENCODED_SECRET_KEY, AWS_BUCKET_NAME), "/mnt/%s" % MOUNT_NAME)

In [2]:
#Add Graphframes library to databricks
from graphframes import *

In [3]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
inputPath = "dbfs:/mnt/yelp-mount"

reviewdf = spark.read.json(inputPath+"/review.json")

In [4]:
business = sqlContext.read.json("/FileStore/tables/filtered_business.json")
business.registerTempTable("business")

In [5]:
%sql select * from business

In [6]:
userdf = sqlContext.read.json("/FileStore/tables/user.json")
userdf.registerTempTable('allUserTable')

In [7]:
reviewdf.registerTempTable('reviewTable')

In [8]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *

# sc is an existing SparkContext.
sqlContext = SQLContext(sc)
filteredUserDf = sqlContext.sql("select * from allUserTable where user_id in (select user_id from reviewTable where business_id in (select business_id from business))")

In [9]:

userdf_exploded = filteredUserDf.withColumn('friends', explode('friends'))
userdf_exploded.registerTempTable("userTable")

In [10]:
%sql select * from userTable

In [11]:
userVertices = userdf_exploded.withColumnRenamed("user_id", "id").distinct()
friendsEdges = userdf_exploded.withColumnRenamed("user_id", "src").withColumnRenamed("friends", "dst")
  
  
Friendgraph = GraphFrame(userVertices,  friendsEdges)


In [12]:
display(userVertices)

In [13]:
display(Friendgraph.vertices)

In [14]:
display(Friendgraph.inDegrees)


In [15]:
results = Friendgraph.pageRank(resetProbability=0.15, maxIter=10)

In [16]:
display(results.vertices.select("id", "pagerank"))

In [17]:
user_MaxReviewCount = Friendgraph.vertices.groupBy().max("review_count")
display(user_MaxReviewCount)

In [18]:
findUsersLink = Friendgraph.bfs("review_count > 2000", "name ='Edward'", "fans > 50" )
display(findUserLink)

In [19]:
results = Friendgraph.find("(A)-[]->(B); (B)-[]->(C); !(A)-[]->(C)")

# Filter out loops (with DataFrame operation)

results = results.filter("A.id != C.id")

# Select recommendations for A to follow C

results = results.select("A", "C")

display(results)